In [1]:
import json
import datetime

In [2]:
data = {
    'latitude' : 55.642280, 
    'longitude' : 37.568376
}

file_path = "data/testdata/data.json"

# Открываем файл и сохраняем данные в формате JSON
with open(file_path, "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

In [3]:
import tensorflow as tf
import numpy as np

In [4]:
from PIL import Image
import cv2

In [5]:
model = tf.saved_model.load('centernet_resnet50_v1_fpn_512x512_coco17_tpu-8/saved_model')


In [7]:
for key, value in model.signatures.items():
    print(f"Signature key: {key}")
    print(value.structured_input_signature)
    print(value.structured_outputs)


Signature key: serving_default
((), {'input_tensor': TensorSpec(shape=(1, None, None, 3), dtype=tf.uint8, name='input_tensor')})
{'detection_classes': TensorSpec(shape=(1, 100), dtype=tf.float32, name='detection_classes'), 'num_detections': TensorSpec(shape=(1,), dtype=tf.float32, name='num_detections'), 'detection_boxes': TensorSpec(shape=(1, 100, 4), dtype=tf.float32, name='detection_boxes'), 'detection_scores': TensorSpec(shape=(1, 100), dtype=tf.float32, name='detection_scores')}


Signature key: serving_default
((), {'input_tensor': TensorSpec(shape=(1, None, None, 3), dtype=tf.uint8, name='input_tensor')})
{'detection_classes': TensorSpec(shape=(1, 100), dtype=tf.float32, name='detection_classes'), 'num_detections': TensorSpec(shape=(1,), dtype=tf.float32, name='num_detections'), 'detection_boxes': TensorSpec(shape=(1, 100, 4), dtype=tf.float32, name='detection_boxes'), 'detection_scores': TensorSpec(shape=(1, 100), dtype=tf.float32, name='detection_scores')}

(
    (), 
    {
        'input_tensor': 
            TensorSpec(
                shape=(1, None, None, 3), 
                dtype=tf.uint8, name='input_tensor'
                )
    }   
)
{
    'detection_classes': 
        TensorSpec(
            shape=(1, 100), 
            dtype=tf.float32, 
            name='detection_classes'
            ), 

    'detection_keypoint_scores': 
        TensorSpec(
            shape=(1, 100, 17), 
            dtype=tf.float32, 
            name='detection_keypoint_scores'
            ), 

    'detection_keypoints': 
        TensorSpec(
            shape=(1, 100, 17, 2), 
            dtype=tf.float32, 
            name='detection_keypoints'
            ), 

    'num_detections': 
        TensorSpec(
            shape=(1,), 
            dtype=tf.float32, 
            name='num_detections'
            ), 

    'detection_boxes': 
        TensorSpec(
            shape=(1, 100, 4), 
            dtype=tf.float32, 
            name='detection_boxes'
            ), 

    'detection_scores': 
        TensorSpec(
            shape=(1, 100), 
            dtype=tf.float32, 
            name='detection_scores'
            )
}

In [8]:
image_path = 'data/testdata/photo.jpg'
image = Image.open(image_path)

# Если изображение в формате grayscale, преобразуйте его в RGB
if image.mode != 'RGB':
    image = image.convert('RGB')

# Преобразуем изображение в массив NumPy с типом uint8
image_array = np.array(image, dtype=np.uint8)

# Добавляем измерение для батча
input_tensor = np.expand_dims(image_array, axis=0)

In [9]:
infer = model.signatures['serving_default']

# Выполнение предсказания
output = infer(tf.convert_to_tensor(input_tensor))

2024-11-27 17:39:11.683540: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:933] Skipping loop optimization for Merge node with control input: StatefulPartitionedCall/Assert_3/AssertGuard/branch_executed/_405


In [10]:
detection_classes = output['detection_classes'].numpy()
detection_scores = output['detection_scores'].numpy()
detection_boxes = output['detection_boxes'].numpy()
num_detections = int(output['num_detections'].numpy()[0])

# Вывод результатов для каждого обнаруженного объекта
for i in range(num_detections):
    print(f"Object {i + 1}:")
    print(f" - Class: {detection_classes[0][i]}")
    print(f" - Score: {detection_scores[0][i]}")
    print(f" - Box: {detection_boxes[0][i]}")

Object 1:
 - Class: 1.0
 - Score: 0.8882125020027161
 - Box: [0.2523547  0.00727479 0.9889239  0.9620599 ]
Object 2:
 - Class: 82.0
 - Score: 0.07202603667974472
 - Box: [0.2455109  0.02137268 0.9952738  1.        ]
Object 3:
 - Class: 82.0
 - Score: 0.0702875629067421
 - Box: [0.24958754 0.0039744  0.9796387  0.90648305]
Object 4:
 - Class: 28.0
 - Score: 0.03964680805802345
 - Box: [0.25027648 0.00995994 0.9794922  0.9397285 ]
Object 5:
 - Class: 90.0
 - Score: 0.035707224160432816
 - Box: [0.9894204  0.8724267  1.         0.91070306]
Object 6:
 - Class: 70.0
 - Score: 0.030567189678549767
 - Box: [0.98810744 0.88207525 1.         0.9207747 ]
Object 7:
 - Class: 62.0
 - Score: 0.03017934039235115
 - Box: [0.8877265  0.17881632 0.99543446 0.7679743 ]
Object 8:
 - Class: 1.0
 - Score: 0.029659777879714966
 - Box: [0.07979149 1.         0.97447807 1.        ]
Object 9:
 - Class: 1.0
 - Score: 0.02830175869166851
 - Box: [0.        1.        0.8491435 1.       ]
Object 10:
 - Class: 82.0

In [ ]:
# 1. Загрузка изображения с использованием Pillow
image_path = 'data/testdata/photo.jpg'
image = Image.open(image_path)

# Преобразование изображения в RGB (если оно не RGB)
if image.mode != 'RGB':
    image = image.convert('RGB')

# Преобразование изображения в массив NumPy с типом uint8
image_np = np.array(image, dtype=np.uint8)

# Преобразование изображения в формат BGR для использования с OpenCV
image_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

# Преобразование изображения в массив NumPy с типом uint8 и добавление измерения для батча
input_tensor = np.expand_dims(image_np, axis=0)

# 2. Загрузка модели и выполнение предсказания
infer = model.signatures['serving_default']

# Выполнение предсказания
output = infer(tf.convert_to_tensor(input_tensor))

# 3. Извлечение результатов модели
# Извлечение выходных данных
detection_boxes = output['detection_boxes'].numpy()[0]  # (1, 100, 4) -> (100, 4)
detection_classes = output['detection_classes'].numpy()[0]  # (1, 100) -> (100,)
detection_scores = output['detection_scores'].numpy()[0]  # (1, 100) -> (100,)
num_detections = int(output['num_detections'].numpy()[0])

# 4. Отображение результатов на изображении с использованием OpenCV
for i in range(num_detections):
    score = detection_scores[i]
    
    # Отображаем только объекты с уверенностью выше порога (например, 0.5)
    if score >= 0.5:
        # Координаты ограничивающей рамки в формате [ymin, xmin, ymax, xmax]
        box = detection_boxes[i]
        ymin, xmin, ymax, xmax = box

        # Поскольку координаты заданы в нормализованном виде (в диапазоне [0, 1]),
        # нужно преобразовать их в координаты изображения
        (im_height, im_width, _) = image_np.shape
        (xmin, xmax, ymin, ymax) = (int(xmin * im_width), int(xmax * im_width), int(ymin * im_height), int(ymax * im_height))

        # Рисуем ограничивающую рамку на изображении
        cv2.rectangle(image_bgr, (xmin, ymin), (xmax, ymax), color=(0, 255, 0), thickness=2)

        # Отображаем класс и оценку уверенности над рамкой
        label = f'Class: {int(detection_classes[i])}, Score: {score:.2f}'
        cv2.putText(image_bgr, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# 5. Отображение изображения с найденными объектами
cv2.imshow('Detected Objects', image_bgr)
cv2.waitKey(0)
cv2.destroyAllWindows()
